In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [13]:
sentence_length = 110

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS","True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS","True") == "True"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=False
pin_commits=False


In [15]:
from tooling.config import Experiment, Transformation

base_experiment_config = Experiment(
    name="Base Config", iterations=1, force=False
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Activity",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Interaction_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

In [16]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)
hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

05:11:16 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Activity': 1, 'Domain_Level': 2, 'Interaction_Level': 3, 'System_Level': 4}
05:11:16 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model

In [17]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production: BiLSTM Train for Staged Application"

bilstm_config = BiLSTM(sentence_length=sentence_length)

bilstm_cfg = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bilstm import bilstm
    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit = pin_commits)

print(bilstm_run_id)

print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

05:11:16 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 110
  batch_size: 32
  number_epochs: 32
  verbose: 1
  weighted_classes: false
  learning_rate: 0.0001
experiment:
  name: 'Production: BiLSTM Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

05:11:16 WARNING:Experiment was already run, aborting


5e8af7ab5e4e4130890d3a77c57ef305
mlflow-artifacts:/32/29b05fc08c354b1b8d999d8619a94c1a/artifacts


PosixPath('src/service/models/bilstm')

## Train SNER First Stage Model

In [18]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production: SNER Train for Staged Application"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner
    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit = pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

05:11:18 INFO:
sner:
  type: SNER
experiment:
  name: 'Production: SNER Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

05:11:18 WARNING:Experiment was already run, aborting


mlflow-artifacts:/32/29b05fc08c354b1b8d999d8619a94c1a/artifacts
da407759ccc948b39f21560b8b3633ed


PosixPath('src/service/models/sner.ser.gz')

## Train BERT First Stage Model

In [19]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production: BERT_1 Train for Staged Application"

bert_1_config = BERT(max_len=sentence_length)

bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit = pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

05:11:19 INFO:
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT_1 Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Activity
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

05:11:19 WARNING:Experiment was already run, aborting


29bb8ff8724b4c7689ddc4f86bc27a7e


PosixPath('src/service/models/bert_1')

## Train BERT Second Stage Model for BERT First Stage

In [20]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bert_config = StagedBERT(max_len=sentence_length)

bert_2_bert_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bert_config,
        experiment=bert_2_bert_experiment_config,
        transformation=label_transformation_config,
        first_model_bert=bert_1_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit = pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

05:11:35 INFO:
first_model_bert:
  bert:
    model: bert-base-cased
    type: BERT
    max_len: 110
    train_batch_size: 32
    validation_batch_size: 32
    number_epochs: 5
    learning_rate_bert: 2.0e-05
    learning_rate_classifier: 0.01
    weight_decay: 0.01
    weighted_classes: false
  experiment:
    name: 'Production: BERT_1 Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_bilstm: null
first_model_sner: null
bert:
  m

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

05:11:57 INFO:Loading Model
05:11:58 INFO:Using device: mps
05:11:58 INFO:Creating hint column


Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

...

05:15:06 INFO:Logged iteration result res.precision=0.7055078197135416 res.recall=0.6385707882619516


{'eval_loss': 0.35849639773368835, 'eval_step': 8, 'eval_precision': 0.7055078197135416, 'eval_recall': 0.6385707882619516, 'eval_label_count': 10, 'eval_runtime': 2.1391, 'eval_samples_per_second': 125.751, 'eval_steps_per_second': 4.207, 'epoch': 5.0}
{'train_runtime': 147.5138, 'train_samples_per_second': 36.437, 'train_steps_per_second': 1.152, 'train_loss': 0.30282772288602944, 'epoch': 5.0}


05:15:12 INFO:Logged iteration result res.precision=0.7253149408241828 res.recall=0.6636970557828414
05:15:12 INFO:Logged iteration result res.precision=0.7253149408241828 res.recall=0.6636970557828414
05:15:12 INFO:Finished iteration=0
05:15:12 INFO:Logging model artifact (might take a while)
05:16:49 INFO:Breaking early after iteration=0 of 5 folds
05:16:50 INFO:Logged experiment result res.mean_precision=0.7253149408241828 res.mean_recall=0.6636970557828414
05:16:50 INFO:Left mlflow context


829e2947b822448eb54b9914a42661cb


PosixPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage

In [21]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_sner_config = StagedBERT(max_len=sentence_length)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit = pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

05:17:07 INFO:
first_model_bert: null
first_model_bilstm: null
first_model_sner:
  sner:
    type: SNER
  experiment:
    name: 'Production: SNER Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
  layers: 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


05:17:08 INFO:Using device: mps
05:17:08 INFO:Found existing run with run_id: da407759ccc948b39f21560b8b3633ed matching the configuration
05:17:08 INFO:Downloading run model from mlflow-artifacts:/33/da407759ccc948b39f21560b8b3633ed/artifacts/0_model.ser.gz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


05:17:09 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
05:17:10 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
05:17:10 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
05:17:12 INFO:Dataset Labels: transformed_dataset['labels']=['0', 'System_Level', 'Domain_Data', 'System_Function', 'Workspace', 'Stakeholder', 'Task', 'Interaction_Data', 'Interaction', 'Activity']
05:17:12 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
05:17:12 INFO:Configured maximal token sequence length: max_len = 110
05:17:14 INFO:Created fold datasets for fold: 0, store

...

05:19:47 INFO:Logged iteration result res.precision=0.7248829529708132 res.recall=0.6380788324940454


{'eval_loss': 0.35927027463912964, 'eval_step': 8, 'eval_precision': 0.7248829529708132, 'eval_recall': 0.6380788324940454, 'eval_label_count': 10, 'eval_runtime': 2.1524, 'eval_samples_per_second': 124.976, 'eval_steps_per_second': 4.181, 'epoch': 5.0}
{'train_runtime': 150.8063, 'train_samples_per_second': 35.642, 'train_steps_per_second': 1.127, 'train_loss': 0.24476580900304457, 'epoch': 5.0}


05:19:53 INFO:Logged iteration result res.precision=0.7084352194004455 res.recall=0.6422433651162286
05:19:53 INFO:Logged iteration result res.precision=0.7084352194004455 res.recall=0.6422433651162286
05:19:53 INFO:Finished iteration=0
05:19:53 INFO:Logging model artifact (might take a while)
05:21:31 INFO:Breaking early after iteration=0 of 5 folds
05:21:32 INFO:Logged experiment result res.mean_precision=0.7084352194004455 res.mean_recall=0.6422433651162286
05:21:32 INFO:Left mlflow context


92fa04894b354683b8c90b755ceeac69


PosixPath('src/service/models/bert_2_sner')

## Train BERT Second Stage Model for BILSTM First Stage

In [22]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bilstm_config = StagedBERT(max_len=sentence_length)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit = pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

05:21:49 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 110
    batch_size: 32
    number_epochs: 32
    verbose: 1
    weighted_classes: false
    learning_rate: 0.0001
  experiment:
    name: 'Production: BiLSTM Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Activity
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_sner: null
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


05:21:49 INFO:Using device: mps
05:21:49 INFO:Found existing run with run_id: 5e8af7ab5e4e4130890d3a77c57ef305 matching the configuration
05:21:49 INFO:Downloading run model from mlflow-artifacts:/32/5e8af7ab5e4e4130890d3a77c57ef305/artifacts/0_model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


05:21:51 INFO:loading projection weights from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz
05:22:26 INFO:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 100) matrix of type float32 from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-02T17:22:26.990406', 'gensim': '4.3.1', 'python': '3.11.4 (main, Jun 20 2023, 19:14:10) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'load_word2vec_format'}
05:22:27 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
05:22:27 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
05:22:27 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67

...

05:25:22 INFO:Logged iteration result res.precision=0.7033692947095227 res.recall=0.639471981999616


{'eval_loss': 0.35459452867507935, 'eval_step': 8, 'eval_precision': 0.7033692947095227, 'eval_recall': 0.639471981999616, 'eval_label_count': 10, 'eval_runtime': 2.0613, 'eval_samples_per_second': 130.497, 'eval_steps_per_second': 4.366, 'epoch': 5.0}
{'train_runtime': 146.5491, 'train_samples_per_second': 36.677, 'train_steps_per_second': 1.16, 'train_loss': 0.3198078155517578, 'epoch': 5.0}


05:25:28 INFO:Logged iteration result res.precision=0.7242304606491978 res.recall=0.6624658589748673
05:25:28 INFO:Logged iteration result res.precision=0.7242304606491978 res.recall=0.6624658589748673
05:25:28 INFO:Finished iteration=0
05:25:28 INFO:Logging model artifact (might take a while)
05:27:04 INFO:Breaking early after iteration=0 of 5 folds
05:27:05 INFO:Logged experiment result res.mean_precision=0.7242304606491978 res.mean_recall=0.6624658589748673
05:27:06 INFO:Left mlflow context


a365d209241245779cdb960b02ad8cee


PosixPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model

In [23]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production: BERT Train for E2E Application"

bert_config = BERT(max_len=sentence_length)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit = pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

05:27:23 INFO:
bert:
  model: bert-base-cased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT Train for E2E Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

2023/08/02 17:27:23 INFO mlflow.tracking.fluent: Experiment with name 'Production: BERT Train for E2E Application' does not exist. Creating a new experiment.
05:27:24 INFO:New experim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


05:27:24 INFO:Using device: mps
05:27:24 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
05:27:24 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
05:27:24 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
05:27:27 INFO:Dataset Labels: transformed_dataset['labels']=['0', 'System_Level', 'Domain_Data', 'System_Function', 'Workspace', 'Stakeholder', 'Task', 'Interaction_Data', 'Interaction', 'Activity']
05:27:27 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
05:27:27 INFO:Configured maximal token sequence length: max_len = 110
05:27:29 INFO:Created 

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

...

05:29:52 INFO:Logged iteration result res.precision=0.6971341553968405 res.recall=0.6398233761687562


{'eval_loss': 0.35496285557746887, 'eval_step': 8, 'eval_precision': 0.6971341553968405, 'eval_recall': 0.6398233761687562, 'eval_label_count': 10, 'eval_runtime': 2.0875, 'eval_samples_per_second': 128.863, 'eval_steps_per_second': 4.311, 'epoch': 5.0}
{'train_runtime': 145.2095, 'train_samples_per_second': 37.015, 'train_steps_per_second': 1.171, 'train_loss': 0.3215824800379136, 'epoch': 5.0}


05:29:58 INFO:Logged iteration result res.precision=0.7117798220201934 res.recall=0.6560636405448642
05:29:58 INFO:Logged iteration result res.precision=0.7117798220201934 res.recall=0.6560636405448642
05:29:58 INFO:Finished iteration=0
05:29:58 INFO:Logging model artifact (might take a while)
05:31:35 INFO:Breaking early after iteration=0 of 5 folds
05:31:36 INFO:Logged experiment result res.mean_precision=0.7117798220201934 res.mean_recall=0.6560636405448642
05:31:36 INFO:Left mlflow context


e77fb13ae0e647878961516031b007a2


PosixPath('src/service/models/bert')